In [53]:
import numpy as np
from scipy.optimize import minimize

# ##############################################################################
# LoadData takes the file location for the yacht_hydrodynamics.data and returns
# the data set partitioned into a training set and a test set.
# the X matrix, deal with the month and day strings.
# Do not change this function!
# ##############################################################################
def loadData(df):
    data = np.loadtxt(df)
    Xraw = data[:,:-1]
    # The regression task is to predict the residuary resistance per unit weight of displacement
    yraw = (data[:,-1])[:, None]
    X = (Xraw-Xraw.mean(axis=0))/np.std(Xraw, axis=0)
    y = (yraw-yraw.mean(axis=0))/np.std(yraw, axis=0)

    ind = range(X.shape[0])
    test_ind = ind[0::4] # take every fourth observation for the test set
    train_ind = list(set(ind)-set(test_ind))
    X_test = X[test_ind]
    X_train = X[train_ind]
    y_test = y[test_ind]
    y_train = y[train_ind]

    return X_train, y_train, X_test, y_test

# ##############################################################################
# Returns a single sample from a multivariate Gaussian with mean and cov.
# ##############################################################################
def multivariateGaussianDraw(mean, cov):
    
    sample = np.zeros((mean.shape[0], )) # This is only a placeholder
    # Task 2:
    # TODO: Implement a draw from a multivariate Gaussian here
    
    sample = np.random.multivariate_normal(mean, cov, 1)

    # Return drawn sample
    return sample

# ##############################################################################
# RadialBasisFunction for the kernel function
# k(x,x') = s2_f*exp(-norm(x,x')^2/(2l^2)). If s2_n is provided, then s2_n is
# added to the elements along the main diagonal, and the kernel function is for
# the distribution of y,y* not f, f*.
# ##############################################################################
class RadialBasisFunction():
    def __init__(self, params):
        self.ln_sigma_f = params[0]
        self.ln_length_scale = params[1]
        self.ln_sigma_n = params[2]

        self.sigma2_f = np.exp(2*self.ln_sigma_f)
        self.sigma2_n = np.exp(2*self.ln_sigma_n)
        self.length_scale = np.exp(self.ln_length_scale)

    def setParams(self, params):
        self.ln_sigma_f = params[0]
        self.ln_length_scale = params[1]
        self.ln_sigma_n = params[2]

        self.sigma2_f = np.exp(2*self.ln_sigma_f)
        self.sigma2_n = np.exp(2*self.ln_sigma_n)
        self.length_scale = np.exp(self.ln_length_scale)

    def getParams(self):
        return np.array([self.ln_sigma_f, self.ln_length_scale, self.ln_sigma_n])

    def getParamsExp(self):
        return np.array([self.sigma2_f, self.length_scale, self.sigma2_n])

    # ##########################################################################
    # covMatrix computes the covariance matrix for the provided matrix X using
    # the RBF. If two matrices are provided, for a training set and a test set,
    # then covMatrix computes the covariance matrix between all inputs in the
    # training and test set.
    # ##########################################################################
    def covMatrix(self, X, Xa=None):
        if Xa is not None:
            X_aug = np.zeros((X.shape[0]+Xa.shape[0], X.shape[1]))
            X_aug[:X.shape[0], :X.shape[1]] = X
            X_aug[X.shape[0]:, :X.shape[1]] = Xa
            X=X_aug

        n = X.shape[0]
        covMat = np.zeros((n,n))

        # Task 1:
        # TODO: Implement the covariance matrix here
        
        
        for i in range(n):
            for j in range(n):
                sqdist = np.exp(-0.5 * (1/(self.length_scale)**2)* (np.linalg.norm(X[i,:]-X[j,:]))**2)
                covMat[i,j] = self.sigma2_f *  sqdist
        

        # If additive Gaussian noise is provided, this adds the sigma2_n along
        # the main diagonal. So the covariance matrix will be for [y y*]. If
        # you want [y f*], simply subtract the noise from the lower right
        # quadrant.
        if self.sigma2_n is not None:
            covMat += self.sigma2_n*np.identity(n)

        # Return computed covariance matrix
        return covMat

class GaussianProcessRegression():
    def __init__(self, X, y, k):
        self.X = X
        self.n = X.shape[0]
        self.y = y
        self.k = k
        self.K = self.KMat(self.X)
        self.L = np.linalg.cholesky(self.K)

    # ##########################################################################
    # Recomputes the covariance matrix and the inverse covariance
    # matrix when new hyperparameters are provided.
    # ##########################################################################
    def KMat(self, X, params=None):
        if params is not None:
            self.k.setParams(params)
        K = self.k.covMatrix(X)
        self.K = K
        self.L = np.linalg.cholesky(self.K)
        return K

    # ##########################################################################
    # Computes the posterior mean of the Gaussian process regression and the
    # covariance for a set of test points.
    # NOTE: This should return predictions using the 'clean' (not noisy) covariance
    # ##########################################################################
    def predict(self, Xa):
        mean_fa = np.zeros((Xa.shape[0], 1))
        mean_fa = np.array(mean_fa.flatten())
        cov_fa = np.zeros((Xa.shape[0], Xa.shape[0]))
        # Task 3:
        # TODO: compute the mean and covariance of the prediction
        
        
        #beginning of my solution: ------->
        na = Xa.shape[0]
        params = self.k.getParamsExp()
        Ktotal = self.k.covMatrix(self.X, Xa)
        # Covariance between training sample points (without Gaussian noise)
        Kxx = Ktotal[0:self.n,0:self.n] # + 1 * np.eye(10) if there is  Gaussian noise
        
        # Covariance between training and test points
        Kxs = Ktotal[self.n:self.n+na, 0:self.n]
        
        # Covariance between test points
        Kss = self.k.covMatrix(Xa)  #- params[2] * np.eye((na,na))  #Ktotal[self.n:self.n+na,self.n:self.n+na] #- params[2]@ np.eye((na,na))
        for i in range(Kss.shape[0]):
                Kss[i,i] = Kss[i,i] - params[2]
        
        # The mean of the GP fit (note that @ is matrix multiplcation: A @ B is equivalent to np.matmul(A,B))
        mean = Kxs @ np.linalg.inv(Kxx) @ self.y
        
        for i in range(mean.shape[0]):
            mean_fa[i] = (mean[i])[0]
        # The covariance matrix of the GP fit
        cov_fa = Kss - Kxs @ np.linalg.inv(Kxx) @ Kxs.T
        

        # Return the mean and covariance
        return mean_fa, cov_fa

    # ##########################################################################
    # Return negative log marginal likelihood of training set. Needs to be
    # negative since the optimiser only minimises.
    # ##########################################################################
    def logMarginalLikelihood(self, params=None):
        if params is not None:
            self.KMat(self.X, params)

        mll = 0
        # Task 4:
        # TODO: Calculate the log marginal likelihood ( mll ) of self.y
        
        mll = 0.5* (self.y).T @ np.linalg.solve((self.L).T, np.linalg.solve(self.L, self.y)) + np.sum(np.log((self.L).diagonal())) + 0.5 * self.n * np.log(2 * np.pi)
        mll = mll[0][0]
        # Return mll
        return mll

    # ##########################################################################
    # Computes the gradients of the negative log marginal likelihood wrt each
    # hyperparameter.
    # ##########################################################################
    def gradLogMarginalLikelihood(self, params=None):
        if params is not None:
            K = self.KMat(self.X, params)

        grad_ln_sigma_f = grad_ln_length_scale = grad_ln_sigma_n = 0
        # Task 5:
        # TODO: calculate the gradients of the negative log marginal likelihood
        # wrt. the hyperparameters
        
        beta_ln_length_scale = np.zeros((self.n, self.n))
        beta_ln_sigma_f = np.zeros((self.n, self.n))
        beta_ln_sigma_n = np.zeros((self.n, self.n))
        
        param = self.k.getParamsExp() #np.array([self.sigma2_f, self.length_scale, self.sigma2_n])
        
        
        alpha = np.linalg.solve(self.K, self.y)
        #inv = np.linalg.solve((self.L).T, np.linalg.solve(self.L, np.eye(self.n,self.n)))
        inv = np.linalg.inv(self.K)
        #L = self.L
        for i in range(self.n):
            for j in range(self.n):
                const = (np.linalg.norm(self.X[i]-self.X[j]))**2
                beta_ln_sigma_f[i,j] = 2 * param[0] * np.exp(-0.5 * const/(param[1]**2))
                beta_ln_length_scale[i,j] = (param[0]/(param[1]**2)) * const * np.exp(-0.5 * const/(param[1]**2)) 
                beta_ln_sigma_n[i,j] =  2 * param[2] if (i==j) else 0
        
        #beta_ln_sigma_n =beta_ln_sigma_n @ 
        grad_ln_sigma_f = -0.5 * np.trace((alpha @ alpha.T - inv) @ beta_ln_sigma_f)
        grad_ln_length_scale = -0.5 * np.trace((alpha @ alpha.T - inv) @ beta_ln_length_scale)
        grad_ln_sigma_n = -0.5 * np.trace((alpha @ alpha.T - inv) @ beta_ln_sigma_n)
        

        # Combine gradients
        gradients = np.array([grad_ln_sigma_f, grad_ln_length_scale, grad_ln_sigma_n])

        # Return the gradients
        return gradients

    # ##########################################################################
    # Computes the mean squared error between two input vectors.
    # ##########################################################################
    def mse(self, ya, fbar):
        mse = 0
        # Task 7:
        # TODO: Implement the MSE between ya and fbar
        
        for i in range(ya.shape[0]):
            mse += np.sum((ya[i] - fbar[i])**2)/(ya.shape[0])

        # Return mse
        return mse

    # ##########################################################################
    # Computes the mean standardised log loss.
    # ##########################################################################
    def msll(self, ya, fbar, cov):
        msll = 0
        # Task 7:
        # TODO: Implement MSLL of the prediction fbar, cov given the target ya
        
        param = self.k.getParamsExp()
        sigma = np.diagonal(cov) 
        
        for i in range(ya.shape[0]):
            msll += 0.5 * np.log(2* np.pi * sigma[i]) + ((ya[i] - fbar[i])**2)/(2 * sigma[i])
            
        msll = msll/(ya.shape[0])

        return msll

    # ##########################################################################
    # Minimises the negative log marginal likelihood on the training set to find
    # the optimal hyperparameters using BFGS.
    # ##########################################################################
    def optimize(self, params, disp=True):
        
        res = minimize(self.logMarginalLikelihood, params, method ='BFGS', jac = self.gradLogMarginalLikelihood, options = {'disp':disp})
        return res.x

if __name__ == '__main__':

    np.random.seed(42)

    ##########################
    # You can put your tests here - marking
    # will be based on importing this code and calling
    # specific functions with custom input.
    ##########################

In [54]:
X_train, y_train, X_test, y_test = loadData('yacht_hydrodynamics.data')
params = [0.5*np.log(1.0), np.log(0.1), 0.5*np.log(0.5)]
basis = RadialBasisFunction(params)

In [55]:
GPR = GaussianProcessRegression(X_train, y_train,basis)

In [56]:
basis.covMatrix(X_train).shape

(231, 231)

In [57]:
(basis.covMatrix(X_train, X_test)).shape

(308, 308)

In [58]:
(GPR.KMat(X_train)).shape

(231, 231)

In [59]:
mean, cov = GPR.predict(X_test)

In [60]:
mean

array([-0.02017627, -0.03629425, -0.02006436,  0.0986253 , -0.03975349,
       -0.03181455,  0.01355653, -0.02014108, -0.03581141, -0.01702384,
        0.10249741, -0.03886745, -0.0296727 ,  0.01630726, -0.02002461,
       -0.03570885, -0.01789738,  0.10318099, -0.03951629, -0.03137032,
        0.01214956, -0.01984454, -0.03491483, -0.01720009,  0.09471578,
       -0.03872453, -0.02895737,  0.01178568, -0.02007949, -0.0362516 ,
       -0.02030838,  0.10966427, -0.03936436, -0.02942882,  0.02344937,
       -0.02035499, -0.03644161, -0.01794647,  0.10505718, -0.03924066,
       -0.03029931,  0.02080704, -0.02023602, -0.03643391, -0.01872925,
        0.09951239, -0.03941362, -0.03247497,  0.02107363, -0.02025761,
       -0.0366612 , -0.02080182,  0.12048901, -0.03981227, -0.03322858,
        0.02170264, -0.02035493, -0.03574428, -0.01116493,  0.1086845 ,
       -0.03986022, -0.02925268,  0.01530366, -0.02008186, -0.03577393,
       -0.01579157,  0.13611811, -0.03937929, -0.03050932,  0.02

In [61]:
cov

array([[ 9.98581806e-001, -1.32804949e-006,  3.77246149e-015, ...,
         3.36566683e-158, -9.00239570e-164,  8.50876833e-172],
       [-1.32804949e-006,  9.97163621e-001, -1.32804547e-006, ...,
         3.36458509e-158,  3.36458508e-158, -9.00235343e-164],
       [ 3.77246149e-015, -1.32804547e-006,  9.97163621e-001, ...,
        -9.00235343e-164,  3.36458508e-158,  3.36458508e-158],
       ...,
       [ 3.36566683e-158,  3.36458509e-158, -9.00235343e-164, ...,
         9.97163623e-001, -1.32804547e-006,  3.77245009e-015],
       [-9.00239570e-164,  3.36458508e-158,  3.36458508e-158, ...,
        -1.32804547e-006,  9.97163621e-001, -1.32804547e-006],
       [ 8.50876833e-172, -9.00235343e-164,  3.36458508e-158, ...,
         3.77245009e-015, -1.32804547e-006,  9.97163621e-001]])

In [62]:
GPR.mse(y_test,mean)

0.5397742114834622

In [63]:
GPR.logMarginalLikelihood()

344.11280059438485

In [64]:
GPR.gradLogMarginalLikelihood()
#[19.636336629105244, -119.96975605541138, 21.582215086368706]

[[0.00000000e+000 2.83697769e-001 1.11184378e-004 ... 6.15447964e-311
  0.00000000e+000 0.00000000e+000]
 [2.83697769e-001 0.00000000e+000 2.83697769e-001 ... 1.38682141e-285
  6.15447964e-311 0.00000000e+000]
 [1.11184378e-004 2.83697769e-001 0.00000000e+000 ... 6.65697904e-263
  1.38682141e-285 6.15447964e-311]
 ...
 [6.15447964e-311 1.38682141e-285 6.65697904e-263 ... 0.00000000e+000
  2.83697769e-001 1.11184378e-004]
 [0.00000000e+000 6.15447964e-311 1.38682141e-285 ... 2.83697769e-001
  0.00000000e+000 2.83697769e-001]
 [0.00000000e+000 0.00000000e+000 6.15447964e-311 ... 1.11184378e-004
  2.83697769e-001 0.00000000e+000]]


array([ 39.27007743, -11.97860468,  21.58116714])

In [52]:
GPR.optimize(np.array(params))
# Warning: Desired error not necessarily achieved due to precision loss.
#          Current function value: -160.314039
#          Iterations: 12
#          Function evaluations: 63
#          Gradient evaluations: 53

# array([ 2.81428412,  1.20090598, -3.71012936])

Optimization terminated successfully.
         Current function value: -160.372577
         Iterations: 15
         Function evaluations: 34
         Gradient evaluations: 34


array([ 2.74952692,  1.18679784, -3.70981841])

In [19]:
optimiz.success

True

In [17]:
GPR.msll(y_test, mean, cov)

array([1.18822945])

In [21]:
mean_y, cov_y = GPR.predict(y_test)

In [22]:
GPR.msll(y_test, mean_y, cov_y)

array([1.28313176])

In [20]:
optimiz.x

array([ 1.26936655,  0.43409084, -4.63542499])

In [19]:
np.random.seed(42)
fbar = np.random.multivariate_normal(mean, cov)
GPR.mse(y_test, fbar)

2.0533896295644682

In [20]:
sigma2_f = np.exp(2*0.5*np.log(1.0))
length_scale= np.exp(np.log(0.1))
sigma2_n = np.exp(2*0.5*np.log(0.5))

In [21]:
def covariance(X,length_scale,sigma2_n,sigma2_f):
    n = X.shape[0]
    covMat = np.zeros((n,n))
    for i in range(n):
            for j in range(n):
                sqdist = np.exp(-0.5 * (1/(length_scale)**2)* (np.linalg.norm(X[i,:]-X[j,:]))**2)
                covMat[i,j] = sigma2_f *  sqdist
                
    
    covMat += sigma2_n*np.identity(n)
    
    return covMat

In [40]:
def mll(X,y,length_scale,sigma2_n,sigma2_f):
    n = X.shape[0]
    K = covariance(X,length_scale,sigma2_n,sigma2_f)
    L = np.linalg.cholesky(K)
    mll = 0.5* y.T @ np.linalg.solve(L.T, np.linalg.solve(L, y)) + np.sum(np.log((L).diagonal())) + 0.5 * n * np.log(2 * np.pi)
    
    return mll

In [23]:
covariance(X_train,length_scale,sigma2_n,sigma2_f).shape

(231, 231)

In [55]:
h = 1e-4
sigma2_f_h = np.exp(2*(0.5*np.log(1.0) + h))
length_scale_h = np.exp(np.log(0.1) + h)
sigma2_n_h = np.exp(2*(0.5*np.log(0.5) +h))

Gsigmaf_h = mll(X_train,y_train,length_scale,sigma2_n, sigma2_f_h)
Gsigmaf = mll(X_train,y_train,length_scale,sigma2_n,sigma2_f)

In [56]:
(Gsigmaf_h - Gsigmaf)/h

array([[39.27919681]])

In [57]:
mll(X_train,y_train,length_scale,sigma2_n,sigma2_f)

array([[344.11280059]])

In [58]:
Gsigman_h = mll(X_train,y_train,length_scale,sigma2_n_h,sigma2_f)
Gsigman = mll(X_train,y_train,length_scale,sigma2_n,sigma2_f)

In [59]:
(Gsigman_h - Gsigman)/h

array([[21.58437356]])

In [60]:
Gl_h = mll(X_train,y_train,length_scale_h,sigma2_n,sigma2_f)
Gl = mll(X_train,y_train,length_scale,sigma2_n,sigma2_f)

In [61]:
(Gl_h - Gl)/h

array([[-11.98104196]])

In [62]:
[((Gsigmaf_h - Gsigmaf)/h)[0][0], ((Gl_h - Gl)/h)[0][0], ((Gsigman_h - Gsigman)/h)[0][0]]

[39.27919681075309, -11.981041961348637, 21.584373556606806]

In [17]:
np.random.multivariate_normal(mean, cov, 1)

array([[-0.37635738,  0.68915205,  1.16231755,  0.85475584,  0.26644606,
         0.26615192,  0.80777712, -2.00578654,  0.54468847,  1.25883852,
         0.1722827 , -0.0674602 , -2.17597851,  0.8040508 , -0.53632586,
         1.67343025, -1.21788792, -1.26104046, -0.28439741,  0.33530352,
         0.27491905, -0.31937151, -0.81590689,  0.25338032,  0.23529457,
         0.79912838, -0.14011833, -1.13177478, -1.49853229, -1.75171792,
        -0.00393174, -0.06600905, -1.30493592,  0.71343701,  0.73793125,
        -0.52145806,  0.02721881,  0.66271595, -0.09418827,  1.07576977,
        -0.15617355, -1.95761762, -0.24096041, -1.3272322 , -0.62485853,
         0.1174323 , -0.37311705, -0.19361842,  0.50230348, -0.3480056 ,
         0.60499786,  0.37096258,  0.57188482,  1.0483519 ,  0.08768342,
        -0.6052435 , -0.93568761, -0.65403634,  0.56360926,  2.26477679,
        -0.18122414,  0.68827425,  0.70304126,  0.07292968, -0.01923004,
        -1.82706165, -0.79143402,  0.73965989,  0.5